In [ ]:
from sympy import *

In [ ]:
# Matrixmultiplikation zweier sympy Matrizen
def mult(A, B):
    # get dimensions of A and B
    [dimA, dimB] = [shape(A), shape(B)]
    if dimA[1] != dimB[0]:
        return None
    else:
        C = zeros(dimA[0], dimB[1])
        # Rows times columns
        for i in range(0, dimA[0]):
            for j in range(0, dimB[1]):
                for k in range(0, dimA[1]):
                    C[i,j] += A[i,k]*B[k,j]
        # Simplify expressions before return
        return simplify(C)

In [ ]:
# Multiplication with elementary matrices
# func is the type of row operation to be performed
# z1 and z2 are row numbers
# A is a matrix
def common(func, z1, z2, c, A):
    dim = shape(A)
    E = eye(dim[0])
    # Define conresponding Matrix E depending of the row operation func
    if func == multRow or func == multRowAdd:
        E[z1, z2] = c
    if func == swapRows:
        E[z1,z1] = 0
        E[z2,z2] = 0
        E[z1,z2] = 1
        E[z2,z1] = 1
    B = E*A
    # B is the resulting Matrix after the row operation on A
    B.applyfunc(simplify)
    return B

# Multiplication of row z from matrix A with the constant c
def multRow(z, c, A):
    return common(multRow, z, z, c, A)

# Add c times row z2 to the row z1 of matrix A
def multRowAdd(z1, z2, c, A):
    return common(multRowAdd, z1, z2, c, A)

# Swap the rows z1 and z2 of matrix A
def swapRows(z1, z2, A):
    return common(swapRows, z1, z2, None, A)

# Returns the row echelon form of the matrix A 
# and a list of the columns form the pivot elements
def gauss(A):
    dim = shape(A)
    rows, columns = dim[0], dim[1]
    [i, j] = [0, 0]
    A.applyfunc(simplify)
    # store pivot elements
    pivots = []
    # start in top left corner of the matrix
    while(i < rows and j < columns):
        # try finding pivot element in column j
        if (A[i,j]==0):
            for k in range(i+1, rows):
                if 0 != A[k,j]:
                    A = swapRows(i, k, A)
                    break
        # use pivot element to eliminate below in column j
        if A[i,j] != 0:
            pivots.append(j)
            # normalize A[i,j]
            A = multRow(i, 1/A[i,j], A)
            for k in range(i+1, rows):
                # eliminate A[k,j]
                A = multRowAdd(k, i, -A[k,j], A)
            # go to next row and column
            i += 1
            j += 1
        else:
            # go to next column
            j += 1
    # eliminate above pivot elements
    i = 0
    for j in pivots:
        for k in range(0, i):
            # eliminate A[k,j]
            A = multRowAdd(k, i, -A[k,j], A)
        i += 1
    return (A, tuple(pivots))

In [ ]:
# Returns the inverse of the matrix A if it exists
# Returns None otherwise
def inverse(A):
    dim = shape(A)
    if dim[0] != dim[1]:
        return None
    else:
        n = dim[0]
        # For calculating the inverse, the unitary matrix is appended after A
        A = Matrix([[A, eye(n)]])
        G = gauss(A)
        # G[1] is the list of pivots
        # A invertible is equivalent to G[1] = [0, 1, ... , n-1]
        if G[1] == tuple(range(0,n)):
            # The first half of G is the unitary matrix
            # return the second half of the Matrix G
            return (gauss(A)[0])[:, list(range(n, 2*n))]
        else:
            return None 

In [ ]:
# return the determinant of the matrix A
def laplace(A):
    dim = shape(A)
    # Square matrix is needed
    if dim[0] != dim[1]:
        return None
    else:
        # We use recursion on the dimension to calculate the determinant
        n = dim[0]
        if n == 1:
            return A[0,0]
        else:
            detA = 0
            parity = 1
            columns = list(range(0,n))
            # columns = [0, 1, ... , n-1]
            # Laplace expansion on the first row
            for j in range(0,n):
                columns.remove(j)
                # A[1:,columns] is the matrix A without row 1 and column j
                # A[1:,columns] has dimension (n-1)x(n-1)
                c = parity * A[0,j] * laplace(A[1:,columns])
                detA += c
                columns.insert(j,j)
                parity *= -1
            return detA

In [ ]:
# Calculates the list of eigenvalues of matrix A
def eigenVal(A):
    dim = shape(A)
    if dim[0] != dim[1]:
        return None
    else:
        n = dim[0]
        # Subtract the symoblic variable x form the diagonal of A
        x = symbols('x')
        A = A - x*eye(n)
        # Calculate characteristic polynominal
        charPoly = laplace(A)
        # get the roots of the characteristic polynominal
        return solve(charPoly, x)

# Calculates numeric values for the eigenvalues of matrix A 
def eigenValNumeric(A):
    return list(map(lambda x: x.evalf(), eigenVal(A)))

<h2>Multiplikation</h2>


In [ ]:
A = Matrix([[1,2,3],[4,5,6],[7,8,9]])
B = Matrix([[0,-1,-2],[-3,-4,-5],[-6,-7,-8]])
pprint(A)
pprint(B)

In [ ]:
C = mult(A,B)
pprint(C)

<h2>Zeilenstufenform (Gauß)</h2>

In [ ]:
A = Matrix([[1, 2, 3, 0],[4, 5, 6, -1],[7, 8, 9, -2],[10,11,12,-4]])
pprint(A)

In [ ]:
res = gauss(A)
pprint(res)

<h2>Inverse Matrix</h2>

In [ ]:
A = Matrix([[1, 2, 3],[4, 5, 6],[7, 8, 0]])
pprint(A)

In [ ]:
B = inverse(A)
pprint(B)

<h2>Determinante</h2>

In [ ]:
A = Matrix([[1, 2, 3],[4, 5, 6],[7, 8, 0]])
pprint(A)

In [ ]:
laplace(A)

<h2>Eigenwerte</h2>

In [ ]:
A = Matrix([[-2, -4, 2],[-2, 1, 0],[4, 2, 5]])
pprint(A)

In [ ]:
res = eigenVal(A)
pprint(res)

In [ ]:
res = eigenValNumeric(A)
pprint(res)

<h2>Äquivalente SymPy Methoden</h2>
<p style="font-size: 20px">
mult(A,B)    <->  A*B <br>
gauss(A)     <->  A.rref() <br>
inverse(A)   <->  A.inv() <br>
laplace(A)   <->  A.det() <br>
eigenVal(A)  <->  A.eigenvals() <br></p>

In [ ]:
#Beispiel Gauselimination

A = Matrix([[1, 2, 3],[4, 5, 6],[7, 8, 9]])
print("A = ")
pprint(A)

print("gauss(A) = ")
B = gauss(A)
pprint(B)

print("builtin sympy Ergebniss = ")
C = A.rref()
pprint(C)

<h2> Hamming Code (7,4) </h2>

In [ ]:
# bitList is a list of 4 bits
# generateHamming return a list of 7 bits, which consists of the 4 input bits
# and 3 additional parity bits 
def generateHamming(bitList):
    # G is a 7x4 Matrix for encoding
    G = Matrix([[1, 1, 0, 1],
                [1, 0, 1, 1],
                [1, 0, 0, 0],
                [0, 1, 1, 1], 
                [0, 1, 0, 0], 
                [0, 0, 1, 0], 
                [0, 0, 0, 1]])
    x = Matrix(bitList)
    y = (G * x) % 2
    # y[0], y[1], y[3] are parity bits
    # y[2], y[4], y[5], y[6] are the bits from bitList
    return list(y)

# bitList is a list of 7 bits
# decodeHamming corrects up to 1 bit flip
# In this case, it returns a tuple consisting of the original
# 4 message bits and a string with information about the corrected bit
def decodeHamming(bitList):
    # H is a 3x7 Matrix for bit flip error checking
    H = Matrix([[1, 0, 1, 0, 1, 0, 1], 
                [0, 1, 1, 0, 0, 1, 1], 
                [0, 0, 0, 1, 1, 1, 1]])
    r = Matrix(bitList)
    zeros3 = zeros(3, 1)
    # The result of the Multiplication of H and r (mod 2)
    # tells if the parity and the message bits are in sync
    if zeros3 == (H * r) % 2:
        return ([r[2], r[4], r[5], r[6]], "Kein Bit korrigiert")
    else:
        # Test all possible bit flips
        for i in range(0, 7):
            e = zeros(7, 1)
            e[i] = 1
            rNew = (r + e) % 2
            # rNew differs from r in the i-th bit
            if zeros3 == (H * rNew) % 2:
                return ([rNew[2], rNew[4], rNew[5], rNew[6]], "Bit "+str(i+1)+" korrigiert")

In [ ]:
generateHamming([1,1,1,1])

In [ ]:
decodeHamming([1, 1, 1, 1, 1, 0, 1])